**Table of contents**<a id='toc0_'></a>    
- [I- LIBRARY INSTALLS](#toc1_)    
- [II- LIBRARY IMPORTS](#toc2_)    
  - [III - DATA PRE-PROCESSING](#toc2_1_)    
      - [III- 1. CREATE THE TEXT PASSAGES (AKA KNOWLEDGE BASE) AS A "TAB-SAPARATED-VALUE" (TSV) FILE](#toc2_1_1_1_)    
      - [III- 2. BREAK DOWN "ORIGINAL" TEXT PASSAGES (AKA KNOWLEDGE BASE) INTO "SHORTER" LENGTH ONES](#toc2_1_1_2_)    
      - [III- 3. EXPLORE DATASET BY LOADING "CHUNKS" FROM SAVED TEXT PASSAGES (IF TOO BIG FOR MEMORY)](#toc2_1_1_3_)    
    - [III- 4. CREATE THE "INPUTS" TENSORS (AKA TOKENS) OF THE KNWOLEDGE BASE DOCUMENTS AS HD5 FILE](#toc2_1_2_)    
    - [III- 5. CREATE EMBEDDINGS FROM INPUTS TENSORS AS HD5 FILE](#toc2_1_3_)    
    - [III- 6. CREATE FAISS INDECES FROM TEXT PASSAGES EMBEDDINGS](#toc2_1_4_)    
    - [III- 7. EXAMPLE: FINDING "BY HAND" SIMILAR DOCUMENTS TO A QUESTION](#toc2_1_5_)    
      - [III- 7.1 EMBED THE USER "PROMPT/QUESTION" INTO A VECTOR](#toc2_1_5_1_)    
      - [III- 7.2 RETRIEVE RELEVANT DOCUMENTS FOR THE USER "PROMPT/QUESTION"](#toc2_1_5_2_)    
      - [III- 7.3 CHECK THE RELEVANT DOCUMENTS RETRIEVED FOR THE USER "PROMPT/QUESTION"](#toc2_1_5_3_)    
      - [III- 7.4 COMPARE BART LLM RESPONSE WITH AND WIHTOUT CONTEXT](#toc2_1_5_4_)    
    - [IV- ORGANIZE TEXT PASSAGES AND THEIR EMBEDDINGS IN THE FORMAT NEEDED BY THE RAG "MODEL"](#toc2_1_6_)    
      - [IV- 11.1 JOIN TEXT PASSAGES AND THEIR EMBEDDINGS IN THE SAME FILE](#toc2_1_6_1_)    
      - [III- 11.2 SPLIT INTO CHUNKS OF APACHE "ARROW" FORMAT FILES THE TEXT PASSAGES AND THEIR EMBEDDINGS](#toc2_1_6_2_)    
  - [IV- "FACEBOOK" RAG "MODEL"](#toc2_2_)    
    - [IV- 1. THE RAG "RETRIEVER"](#toc2_2_1_)    
    - [IV- 2. THE RAG "GENERATOR"](#toc2_2_2_)    
      - [IV- 2.1 RESPONSE BASED ON COSINE SIMILARITY WITH RETRIEVED DOCUMENTS](#toc2_2_2_1_)    
    - [IV- 2.2. RESPONSE BASED ON EUCLIDEAN DISTANCE WITH RETRIEVED DOCUMENTS](#toc2_2_3_)    
    - [IV- 2.3. RESPONSE BASED ON DOT PRODUCT WITH RETRIEVED DOCUMENTS](#toc2_2_4_)    
- [X- APPENDIX](#toc3_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[I- LIBRARY INSTALLS](#toc0_)

In [ ]:
# The PyTorch package is officially named torch on pip

# ! pip install transformers datasets torch 
# ! conda install pytorch torchvision torchaudio pytorch-cuda=12.4 -c pytorch -c nvidia 

# ! pip show torch
# ! conda install faiss-gpu 

# ! conda install -c conda-forge datasets --update-deps
# ! pip install --upgrade datasets

# !pip install pyarrow

# beautifulsoup4 is the name to install via pip
# !pip install beautifulsoup4 

# <a id='toc2_'></a>[II- LIBRARY IMPORTS](#toc0_)

In [1]:
%%time

# Automatically detect and use all available cores
import multiprocessing
import os
import psutil
import shutil

import gzip
import itertools
# Dask processes data in parallel without HAVING TO MANUALLY CODE FOR LOADING IN CHUNKS
import dask.dataframe as dd
from dask.distributed import Client, LocalCluster
from dask_cuda import LocalCUDACluster
import dask_cudf  # GPU-accelerated DataFrames (replaces dask.dataframe)
# DASK ALLOWS "PARALLELIZED" processing OF DATA IN MULTIPLE "CHUNKS" simultaneously ON SEVERAL CPU cores (OR GPU WIRH DASK-CUDA)
#from concurrent.futures import ProcessPoolExecutor
import cudf
import numba
from numba import cuda

import json
import h5py
#import tables # After conda install -c anaconda pytables
import gc
#import zipfile

import requests
import numpy as np
import pandas as pd
import pickle


import time
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

# bs4 is the module of beautifulsoup4 that contains the BeautifulSoup class.
from bs4 import BeautifulSoup

#import pyarrow as pa
#import pyarrow.feather as feather
#import pyarrow.parquet as pq
#import pyarrow.ipc as ipc

# Hugging Face's Transformers library primarily supports RAG models in PyTorch
import torch, torchvision, torchaudio
import torch.nn.functional as F

#  FAISS MEANS "Facebook AI Similarity Search"
# TO CRREATE INDECES ON TEXT KNOWLED BASE
import faiss

# DPR MEANS Dense Passage Retrieval (ENCODE QUERY WITH KNOWLEDGE BASE)
from transformers import AutoTokenizer, DPRQuestionEncoderTokenizer, DPRContextEncoderTokenizer, DPRQuestionEncoder, DPRContextEncoder,\
                        BartTokenizer, BartForConditionalGeneration, RagRetriever, RagTokenizer, RagTokenForGeneration 
from transformers import BartTokenizerFast, DPRQuestionEncoderTokenizerFast

from datasets import load_dataset, Dataset, concatenate_datasets, load_from_disk

print("Current Working Directory:", os.getcwd())


print(f"\nTotal RAM available 'NOW': {psutil.virtual_memory().total / 1e9:.2f} GB\n")

2025-02-17 08:27:44.419255: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-17 08:27:44.986509: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-17 08:27:44.986533: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-17 08:27:44.989168: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-17 08:27:45.231028: I tensorflow/core/platform/cpu_feature_g

Current Working Directory: /mnt/d/ANALYTICS - Continuous Education/RAG

Total RAM available 'NOW': 67.36 GB

CPU times: user 3.79 s, sys: 984 ms, total: 4.77 s
Wall time: 11.5 s


In [ ]:
datasets_storage_on_disk = '/mnt/f/RAG DATASETS'

## <a id='toc2_1_'></a>[III - DATA PRE-PROCESSING](#toc0_)

#### <a id='toc2_1_1_1_'></a>[III- 1. CREATE THE TEXT PASSAGES (AKA KNOWLEDGE BASE) AS A "TAB-SAPARATED-VALUE" (TSV) FILE](#toc0_)

In [6]:
# FUNCTION TO EXTRACT "TEXT PASAGES" FROM AN HTML STRING "COLUMN" THAT MUST "ALREADY" EXIST IN THE DATASET

def extract_text_from_wikipedia_html(document_text, row_number, modulo_for_print):    
    
    paragraphs_as_content_div_find_all_p_is_empty={}
    
    #print(document_text)
    # Parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(document_text, 'html.parser')
    content_div = soup.find_all('p')     
    #print("content_div", content_div, type(content_div))
    
    if content_div is None:                  
          print("Content section 'mw-content-text' is None for Row number:", row_number)          
          text_content = " "
          return text_content        

    cleaned_paragraphs = []

     # Loop through the content and extract text from paragraphs
    for element in content_div:
          #text = element.get_text(strip=True)
          text = element.get_text(strip=False)                          
          #print(text)
          # Only append non-empty text
          if len(text.strip()) > 0:                 
               cleaned_paragraphs.append(text)    

     # Combine all paragraphs into a single string
    text_content = ' '.join(cleaned_paragraphs)
     # Replace newline characters with a space
    text_content = text_content.replace("\n", " ").strip()
     #text_content = text_content.replace("\t", " ")

    if not len(text_content.strip()) >0:
                              
          paragraphs_as_content_div_find_all_p_is_empty[row_number]=( content_div)
          #print(row_number,":", url, "\n", content_div)
          print("Row number:",row_number,"Paragraphs from find('p') is EMPTY:")

     
    if row_number % modulo_for_print == 0:
     print(f"Processed row: {row_number}") 
                   
    
    return text_content


In [7]:
# FUNCTION TO "EFFICIENTLY" CREATE IN "BATCHES" THE TAB-SEPARATED (TSV) FILE THAT WILL CONTAIN THE TEXT PASSAGES (AKA KNOWLEDGE BASE OR CONTEXT)

def batches_append_to_tsv(data_generator, output_path, batch_size, modulo_for_print):  

    batch = []       
    row_num = []                

    # OPTION 1: USE LIST "COMPREHENSION"
    # "chunk_size" WILL BE PROVIDED VIA "data_generator" LATER
    # YOU COULD check if it exists before using it VIA chunk_size = getattr(data_generator, "chunk_size", None)
    # chunk_created_passages_list =[extract_text_from_wikipedia_html(document_text, text_position_in_chunk+ chunk_size*chunk_number, modulo_for_print) 
    #                                                 # 1st "FOR" LOOP 
    #                                                 for chunk_number, chunk in enumerate(data_generator) 
    #                                                 # ONLY WHEN THE CONDTION BELOW IS MET WILL THE NEXT "FOR" LOOP BE ENTERED AND "THUS" RETURN THE FUNCTION
    #                                                 #if chunk_number <= 2  # This acts like a break condition                                                         
    #                                                 for text_position_in_chunk, document_text in enumerate(chunk)                                               
    #                                  ]

    # OPTION 2: JUST USE "FOR" LOOPS
    for chunk_number, chunk in enumerate(data_generator):
          
          for text_position_in_chunk, document_text in enumerate(chunk):
                # "chunk_size" WILL BE PROVIDED VIA "data_generator" LATER
                # YOU COULD check if it exists before using it VIA chunk_size = getattr(data_generator, "chunk_size", None)
                chunk_created_passages_list =[extract_text_from_wikipedia_html(document_text, text_position_in_chunk+ chunk_size*chunk_number, 
                                                                               modulo_for_print)
                                                ]
                
                batch = batch + chunk_created_passages_list  
                row_num.append(text_position_in_chunk + chunk_size*chunk_number)

                     
          
          if len(batch) > batch_size:

            #print(row_num)
            #print(len(batch))
            
            data_created = zip(row_num,batch)
            # Filter out tuples where the second element is missing or empty
            data_created = [tup for tup in data_created if tup[1]]            

            # Convert the list to a DataFrame             
            df_batch = pd.DataFrame(data_created) 
            #df.to_csv(output_path, sep='\t', mode='a', header=not pd.io.common.file_exists(output_path), index=False)
            df_batch.to_csv(output_path, sep='\t', mode='a', header=None, index=False)
            
            # Clear the batch and row numbers after writing to the file
            row_num.clear()
            batch.clear()
            chunk_created_passages_list.clear()
            del data_created
            del df_batch
            gc.collect()     
          
          
    if len(batch) > 0:
         
         #print(row_num)
         #print(len(batch))

         data_created = zip(row_num,batch)
         # Filter out tuples where the second element is missing or empty
         data_created = [tup for tup in data_created if tup[1]]            

         # Convert the list to a DataFrame             
         df_batch = pd.DataFrame(data_created) 
         #df.to_csv(output_path, sep='\t', mode='a', header=not pd.io.common.file_exists(output_path), index=False)
         df_batch.to_csv(output_path, sep='\t', mode='a', header=None, index=False)
      
         # Clear the batch and row numbers after writing to the file
         row_num.clear()
         batch.clear()
         chunk_created_passages_list.clear()
         del data_created
         del df_batch
         gc.collect()     

    return 


In [ ]:
# GENERATOR FUNCTION FOR "LOADING" DATA IN CHUNKS FROM A SAVED FILE (HERE ZIPPED)
# USING "itertools.islice"

def chunks_generator_for_gz_zipped_file(data_path, chunk_size):

    with gzip.open(data_path, 'rt', encoding='utf-8') as f:

        # READ ONLY A NUMBER OF ROWS FROM THE SAVED-TO-DISK DATASET THAT IS EQUAL TO "chunk_size"
        #chunk_count = 0 
        for chunk in iter(lambda: list(itertools.islice(f, chunk_size)), []): 
             
            # LIMIT THE NUMBER OF CHUNKS TO READ FROM THE SAVED-TO-DISK DATASET
            #if chunk_count >= 10:  
            #    break            
            
            # YIELD ONLY EXIST WITHIN A FUNCTION
            yield chunk
            #chunk_count += 1 

In [4]:
# DATASET FILE IS (4.4GB): "v1.0-simplified_simplified-nq-train.jsonl.gz". "I" RENAMED IT "google_nq_simplified_dataset_v1.gz" FOUND AT:
# http://storage.cloud.google.com/natural_questions/v1.0-simplified/simplified-nq-train.jsonl.gz

file_path_to_google_nq_dataset = os.path.join(datasets_storage_on_disk, "data",'google_nq_simplified_dataset_v1.gz') 

tsv_file_path_original = os.path.join(datasets_storage_on_disk, "data","text_passages_original.tsv")

In [ ]:
%%time


# RUN THE "batches_append_to_tsv" FUNCTION ABOVE


chunk_size =300
batch_size=4000
modulo_for_print=5000

batches_append_to_tsv( data_generator = chunks_generator_for_gz_zipped_file(data_path = file_path_to_google_nq_dataset, chunk_size = chunk_size),
                       output_path=tsv_file_path_original,
                       batch_size=batch_size,
                       modulo_for_print = modulo_for_print
                       
                       )
                       

In [ ]:
%%time

data_size_initial = 0
#for chunk_df in pd.read_csv(file_path, skiprows=range(1, start_row + 1), chunksize=chunk_size):
for chunk in pd.read_csv(tsv_file_path_original, sep='\t', names=['id', 'text'], chunksize=50000):  
    data_size_initial += len(chunk)

data_size_initial 

CPU times: user 19 s, sys: 3.17 s, total: 22.2 s
Wall time: 1min


307216

#### <a id='toc2_1_1_2_'></a>[III- 2. BREAK DOWN "ORIGINAL" TEXT PASSAGES (AKA KNOWLEDGE BASE) INTO "SHORTER" LENGTH ONES](#toc0_)

* CONSERVE MEANING THROUGH BOTH OVERLAPPING AND CONTEXTUAL ENCODING/DECODING BEFORE ("VIA "AutoTokenizer") CHUNCKING


* Most transformer-based models Have a maximum input size of 512 tokens (chunk_size). With chunk_size=256, stride=128, each chunk is Small enough to fit in the model and Large enough to capture meaningful context

* FOR RAG, "STANDARD" Dense Passage Retrieval (DPR) MODELS process passages of 100-300 tokens to balance efficiency and retrieval quality.

In [11]:

# Most transformer-based models Have a maximum input size of 512 tokens (chunk_size). With chunk_size=256, stride=128, each chunk is Small enough to fit 
# in the model and Large enough to capture meaningful context
# FOR RAG, "STANDARD" Dense Passage Retrieval (DPR) MODELS process passages of 100-300 tokens to balance efficiency and retrieval quality.

def sliding_window_chunking(texts, model_name="bert-base-uncased", chunk_size=256, stride=128):

    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    # Ensure 'texts' is a list. If it's a string, convert it into a list with one element.
    if isinstance(texts, str):  
        texts = [texts]
        
    all_chunks = []  # TO Store all document chunks
    
    for text in texts:  # Loop through each document

        # Tokenize text into IDs
        tokens = tokenizer.encode(text, add_special_tokens=False)  # No CLS/SEP tokens
        #print("token length", len(tokens), "text:",text )

        # Create overlapping chunks. If you split text into fixed chunks of 256 tokens with no overlap, you risk losing contextual meaning at chunk 
        # boundaries. For "stride=128" the first half of each chunk is the end (overlaps with) of the previous chunk
        start = 0
        
        while start < len(tokens):
            chunk = tokens[start : start + chunk_size]
            all_chunks.append(tokenizer.decode(chunk))  # Convert back to text
            #print("chunk", tokenizer.decode(chunk))
            start += stride  # Move window forward
    
    #print("all_chunks LENGTH: ", len(all_chunks), type(all_chunks),"\n", all_chunks)
    
    return all_chunks
 


In [ ]:
# NOT USING "DASK-CUDF" TO PROCESS ON GPU BECAUSE WITH A DASK-CUDF "DATAFRAME", ".map()" EXPECTS A NUMPY OR NUMBA-COMPATIBLE FUNCTION FOR THE FUNCTION
# "sliding_window_chunking()" IN: "df = df.assign(new_col=df["col1"].map(sliding_window_chunking, meta=('new_col', 'object')))" BELOW:

In [ ]:

def break_original_documents_into_smaller_ones(in_path, out_path):


    # Configure Dask "Client" to use all available cores and memory efficiently
    # If you don’t initialize a client, Dask runs in "single-threaded" mode (local scheduler) then Computations still run in parallel, but they are limited to
    # the threads OF A "SINGLE" PROCESSOR. WITH "client", Dask switches to "DISTRIBUTED COMPUTING" WITH MULTIPLE WORKER PROCESSES, NOT just threads

    
    #client.close()
    #cluster.close()

    # Initializing the Client with "client = Client()" automatically creates a local cluster under the hood. no need for cluster = LocalCluster()!!
    client = Client(
    n_workers= multiprocessing.cpu_count(),                # Use all CPU cores as workers
    threads_per_worker=1,             # Use 1 thread per worker (prevents oversubscription)
    #memory_limit="96GB"                # Use ~75% of 128GB RAM to avoid memory pressure. IF OMITTED WILL USE "ALL" AVAILABLE RAM!!!
                        )
    
    print(f"\n{client}\n")



    # Read the file into a Dask DataFrame, assigning explicit column names
    # "dd.read_csv()" loads data "LAZILY" and in parallel. It does not load the entire file into memory at once like Pandas' pd.read_csv
    df = dd.read_csv(in_path, sep="\t", header=None, dtype=str, names=["col0", "col1"])  # Give names to columns


    # Apply my_function to the second column
    df = df.assign(new_col=df["col1"].map(sliding_window_chunking, meta=('new_col', 'object')))


    # Explode the new column to create new rows
    df = df.explode("new_col")


    # Save the result as a new TSV file
    df["new_col"].to_csv(out_path, sep="\t", header=False, index=True, single_file=True)

    #print(f"\nProcessing complete. NEW TSV file created.")

    return




In [6]:
tsv_file_path1 = os.path.join(datasets_storage_on_disk, "data", "text_passages_reduced_length.tsv")

In [ ]:
%%time



break_original_documents_into_smaller_ones(in_path=tsv_file_path_original, out_path=tsv_file_path1)

In [ ]:
%%time

data_size = 0
#for chunk_df in pd.read_csv(file_path, skiprows=range(1, start_row + 1), chunksize=chunk_size):
for chunk in pd.read_csv(tsv_file_path1, sep='\t', names=['id', 'text'], chunksize=50000):  
    data_size += len(chunk)

data_size 

CPU times: user 42.8 s, sys: 1.19 s, total: 44 s
Wall time: 2min 10s


8825328

#### <a id='toc2_1_1_3_'></a>[III- 3. EXPLORE DATASET BY LOADING "CHUNKS" FROM SAVED TEXT PASSAGES (IF TOO BIG FOR MEMORY)](#toc0_)

In [15]:
# GENERATOR FUNCTION FOR "CHUNK-LOADING" DATA FROM A FILE (HERE TSV)

def chunks_generator_for_TSV_file_limited(data_path, chunk_size):

    chunks_read= []
    chunk_count = 0 

    # READ ONLY A NUMBER OF ROWS FROM THE SAVED-TO-DISK DATASET THAT IS EQUAL TO "chunk_size"
    # for chunk_df in pd.read_csv(file_path, skiprows=range(1, start_row + 1), chunksize=chunk_size):
    for chunk in pd.read_csv(data_path, sep='\t', names=['id', 'text'], chunksize=chunk_size):

        chunks_read.append(chunk) 
                  
        # LIMIT THE NUMBER OF CHUNKS TO READ FROM THE SAVED-TO-DISK DATASET
        if chunk_count >= 50:  
                break            
        # THE YIELD COMMAND ONLY EXISTS WITHIN A FUNCTION
        yield chunk
        chunk_count += 1 
    
    return chunks_read



In [ ]:
chunks_read_df = pd.concat(chunks_generator_for_TSV_file_limited(tsv_file_path1, chunk_size=1), ignore_index=True)
print(chunks_read_df.shape)
print(chunks_read_df.head())

sample_chunks = os.path.join(datasets_storage_on_disk, "data","sample_chunks_"+str(chunks_read_df.shape[0])+".tsv")

chunks_read_df.to_csv(sample_chunks, sep='\t', mode='a', header=True, index=False)

# RELEASE MEMORY
chunks_read_df = None
del chunks_read_df
gc.collect()

(50, 2)
   id                                               text
0   0  email marketing is the act of sending a commer...
1   1  customers to purchase something immediately, a...
2   2  blocking out content from emails with filters ...
3   3  real time, and to monitor how effective their ...
4   4  with a few other narrow definitions of transac...


12

### <a id='toc2_1_2_'></a>[III- 4. CREATE THE "INPUTS" TENSORS (AKA TOKENS) OF THE KNWOLEDGE BASE DOCUMENTS AS HD5 FILE](#toc0_)

* An HDF5 (Hierarchical Data Format version 5) file is a "BINARY" file FORMAT used to store large amounts of data efficiently and in a STRUCTURED way.

In [ ]:

# print("\n TRANSFERT of inputs TO Gpu:")
# inputs_tensor = {key: value.to(device) for key, value in inputs.items()}
# print(type(inputs_tensor), len(inputs_tensor))
# inputs_tensor.keys()
# path_to_inputs_tensor =  os.path.join(datasets_storage_on_disk, "data","inputs_tensor.pth")

# TO SAVE AS TENSOR WITH torch.save() USE EXTENSION ".pt" (STANDS FOR PYTORCH) WHICH IS TYPICALLY TO SAVE (AKA SERIALIZED) A "SINGLE" OR A COLLECTION OF 
# TENSORS WHILE EXTENSION ".pth" IS TO SAVE MODELS INCLUDING THEIR ARCHITECTURE ALONG THE WEIGHTS
# PyTorch does NOT support APPENDING to a .pt or .pth file. THUS YOU MUST load the existing file, add the new data, and then save it again. 
# "NOT PREFERRED" BECAUSE WILL EVENTUALLY RUN OUT OF MEMORY!!!!
#torch.save(inputs_tensor, path_to_inputs_tensor)


# TO SAVE AS NUMPY
# Convert tensors to numpy arrays and save them
#inputs_numpy = {key: value.cpu().detach().numpy() for key, value in inputs_tensor.items()}

#path_to_inputs_tensor_as_numpy = os.path.join(datasets_storage_on_disk, "data","inputs_tensor.npy")

# numpy np.save() does NOT support appending data to an existing .npy file AS np.save() overwrites the file AT EACH CALL
#YOU MUST load the existing file, add the new data, and then save it again.
# "NOT PREFERRED" BECAUSE WILL EVENTUALLY RUN OUT OF MEMORY!!!!
#np.save(path_to_inputs_tensor_as_numpy, inputs_numpy)


# PREFERRED IS TO USE HDF5 (Hierarchical Data Format version 5) FORMAT 
# FOR EFFICIENT INCREMENTAL APPENDING AND SAVING DATA

In [ ]:
def create_inputs_tensor_as_numpy(path_to_df, chunk_size, max_length, batch_size, output_path):
    
    # INSTANTIATE THE pre-trained TOKENIZER TO BE APPLIED THE TEXT PASSAGES (AKA KNOWLEDGE BASE )
    trained_model_used_in_context_tokenizer = 'facebook/dpr-ctx_encoder-single-nq-base'

    #tokenizer_passage_encoder = AutoTokenizer.from_pretrained(trained_model_used_in_context_tokenizer) 
    #tokenizer_passage_encoder = DPRQuestionEncoderTokenizer.from_pretrained(trained_model_used_in_context_tokenizer)     
    # REGARDING WARNING AT EXECUTION TIME: "The tokenizer class you load from this checkpoint is not the same type as the class this function is called from"
    # Functionally, both DPRQuestionEncoderTokenizer and DPRContextEncoderTokenizer tokenizers are identical, so it doesn’t impact model performance. 
    tokenizer_passage_encoder = DPRContextEncoderTokenizer.from_pretrained(trained_model_used_in_context_tokenizer) 

    
                                                                            
    batch_of_inputs = []    

    # for chunk_df in pd.read_csv(file_path, skiprows=range(1, start_row + 1), chunksize=chunk_size):                
    for chunk_df in pd.read_csv(path_to_df, sep='\t', names=['id', 'text'], chunksize=chunk_size):          

          # CREATE A LIST WITH AS MANY ELEMENTS AS THE DATAFRAME CHUNK
          passages_list = list(chunk_df['text'])
          #passages_ids = list(chunk_df['id'])


          #print("\n CPU compute of inputs:") 
          # Hugging Face's DPRContextEncoderTokenizer "CANNOT" USE GPU Because Tokenizations are purely CPU-based operations. 
          # IT WILL RETURN A DICTIONARY CONTAINING PyTorch TENSORS 
          inputs = tokenizer_passage_encoder(passages_list, return_tensors='pt', padding=True, max_length=max_length, truncation=True)

          # MANY libraries (like NumPy) READ/WRITE EASILY FROM "HDF5" STORAGE FILE AND NumPy arrays USE "LESS" MEMORY THAN PYTORCH TENSORS, THUS THE 
          # CONVERSION BELOW
          inputs_numpy = {key: value.detach().numpy() for key, value in inputs.items()}
          del inputs
          #print(inputs_numpy.keys())

          batch_of_inputs.append(inputs_numpy)

          if len(batch_of_inputs)>batch_size:
               
               print(len(batch_of_inputs))
               
               # "h5py.File(output_path, 'a')"  OPENS THE FILE IN  "append" mode WITHOUT DELETING EXISTING data OR AUTOMATICALLY 
               # CREATE IF "at output_path"
               with h5py.File(output_path, 'a') as f:

                    # Iterate over each key in the first dictionary to initialize datasets
                    for key in batch_of_inputs[0].keys():

                         #print("key when batch size met", key)
                         # Stack all data for the current key across dictionaries into a single NumPy array
                         print(key)
                         #data_array = np.stack([d[key] for d in batch_of_inputs])
                         data_array = np.concatenate([d[key] for d in batch_of_inputs], axis=0)  # Axis 0 stacks row-wise


                         # Check if the dataset already exists in the file
                         if key in f:

                              # Resize the dataset to accommodate the new data
                              dataset = f[key]
                              dataset.resize((dataset.shape[0] + data_array.shape[0]), axis=0)
                              dataset[-data_array.shape[0]:] = data_array
                         else:
                              # Create a new dataset with expandable dimensions
                              f.create_dataset(key, data=data_array, maxshape=(None,) + data_array.shape[1:])

               # Clear the batch and row numbers after writing to the file
               passages_list.clear()
               del inputs_numpy
               batch_of_inputs.clear()               
               del data_array
               gc.collect()          
       
    if len(batch_of_inputs)>0: 
                         
          print(len(batch_of_inputs))

          # "h5py.File(output_path, 'a')"  OPENS THE FILE IN  "append" mode WITHOUT DELETING EXISTING data OR AUTOMATICALLY 
          # CREATE IF "at output_path"
          with h5py.File(output_path, 'a') as f:

               # Iterate over each key in the first dictionary to initialize datasets
               for key in batch_of_inputs[0].keys():

                    #print("key when batch size NOT met:", key, )
                    # Stack all data for the current key across dictionaries into a single NumPy array
                    print(key)
                    #data_array = np.stack([d[key] for d in batch_of_inputs])
                    data_array = np.concatenate([d[key] for d in batch_of_inputs], axis=0)  # Axis 0 stacks row-wise

                    # Check if the dataset already exists in the file
                    if key in f:

                         # Resize the dataset to accommodate the new data
                         dataset = f[key]
                         dataset.resize((dataset.shape[0] + data_array.shape[0]), axis=0)
                         dataset[-data_array.shape[0]:] = data_array
                    else:
                         # Create a new dataset with expandable dimensions
                         f.create_dataset(key, data=data_array, maxshape=(None,) + data_array.shape[1:])

          # Clear the batch and row numbers after writing to the file
          passages_list.clear()
          del inputs_numpy
          batch_of_inputs.clear()               
          del data_array
          gc.collect() 
                    

In [ ]:
%%time


divisor_value=15816
chunk_size_for_inputs_tensor_as_numpy = divisor_value 
max_length = 256
batch_size_for_inputs_tensor_as_numpy = 10 
path_to_inputs_tensor_as_numpy = os.path.join(datasets_storage_on_disk, "data","inputs_tensors_saved_as_numpy_NEW.h5")
#path_to_inputs_tensor_as_numpy = os.path.join(datasets_storage_on_disk, "data","inputs_tensors_saved_as_numpy.h5")



create_inputs_tensor_as_numpy(path_to_df=tsv_file_path1, 
                                 chunk_size=chunk_size_for_inputs_tensor_as_numpy,
                                 max_length = max_length,
                                 batch_size=batch_size_for_inputs_tensor_as_numpy,
                                 output_path=path_to_inputs_tensor_as_numpy,
                                 )
                            

In [446]:
path_to_inputs_tensor_as_numpy = os.path.join(datasets_storage_on_disk, "data","inputs_tensors_saved_as_numpy_NEW.h5")

In [ ]:

with h5py.File(path_to_inputs_tensor_as_numpy, 'r') as f:
    for key in f.keys():
        dataset = f[key]
        print(f"{key} : {dataset.shape}")


input_tensors_created = dataset.shape[0]

print(f"\nTOTAL TENSORS CREATED (AKA TOKENIZED PASSAGES): {input_tensors_created}")
print(f"\nTOTAL PASSAGES ('DATA SIZE') SUBMITTED: {data_size}")
print(f"\nMISSING TENSORS FOR PASSAGES: {data_size-input_tensors_created}")

attention_mask : (8825328, 256)
input_ids : (8825328, 256)
token_type_ids : (8825328, 256)

TOTAL TENSORS CREATED (AKA TOKENIZED PASSAGES): 8825328

TOTAL PASSAGES ('DATA SIZE') SUBMITTED: 8825328

MISSING TENSORS FOR PASSAGES: 0


### <a id='toc2_1_3_'></a>[III- 5. CREATE EMBEDDINGS FROM INPUTS TENSORS AS HD5 FILE](#toc0_)
 * An HDF5 (Hierarchical Data Format version 5) file is a binary file format used to store large amounts of data efficiently and in a STRUCTURED way.

In [52]:
device = torch.device('cuda:0')
device

device(type='cuda', index=0)

In [53]:
def create_text_passage_embeddings(file_source_path, chunks_grabbed_from_source_h5_file, batch_size, output_path):    
                                                                           
    trained_model_used_in_context_encoder = 'facebook/dpr-ctx_encoder-single-nq-base'
    # INSTANTIATE THE pre-trained EMBEDDING MODEL FOR KNOWLEDGE BASE (AKA PASSAGES)
    #model_passages_encoder = DPRQuestionEncoder.from_pretrained(trained_model_used_in_context_encoder)   
    model_passages_encoder = DPRContextEncoder.from_pretrained(trained_model_used_in_context_encoder)   

    model_passages_encoder = model_passages_encoder.to(device)
    
    batch_of_embeddings = []    

    # Open the HDF5 file
    with h5py.File(file_source_path, 'r') as f:          

    # Iterate over the keys (assuming the keys are 'input_ids', 'attention_mask', etc.)
    # DOES "NOT" REALLY LOOP BECAUSE "f.keys()[0]" IS JUST A LIST OF 1 ELEMENTS!!!! BAD PROGRAMMING BUT GIVES CORRECT RESULT
     for key in list(f.keys())[:1]:
          
          
          dataset = f[key]
          total_chunks_in_source_h5_file= dataset.shape[0]
          print(f"\n THESE ARE THE TOTAL ROWS (CHUNKS) IN INPUT HD5 FILE : {total_chunks_in_source_h5_file} with dataset shape {dataset.shape}")
          #print(f"\n THESE ARE THE keys : {f.keys()}")
          # Load data in chunks
          for start in range(0, total_chunks_in_source_h5_file, chunks_grabbed_from_source_h5_file):

               end = min(start + chunks_grabbed_from_source_h5_file, total_chunks_in_source_h5_file)
               #chunk_data = dataset[start:end]
               #print(chunk_data.keys())  
               
               # GRAB INPUTS SAVED AS NUMPY
              
               
               input_ids = f['input_ids'][start:end]             
               attention_mask = f['attention_mask'][start:end]            
               token_type_ids = f['token_type_ids'][start:end]  
               

               # CONVERT INPUTS SAVED AS NUMPY TO TENSORS AND THEN MOVE THEM TO GPU BEFORE PERFORMING THE EMBEDDING ON GPU
               inputs = {'input_ids': torch.tensor(input_ids.reshape(-1, max_length)).to(device),
                         'attention_mask': torch.tensor(attention_mask.reshape(-1, max_length)).to(device),
                         'token_type_ids':torch.tensor(token_type_ids.reshape(-1, max_length)).to(device),
                         }
  
               # ON CPU
               # inputs = {'input_ids': torch.tensor(input_ids),
               #           'attention_mask': torch.tensor(attention_mask),
               #           'token_type_ids':torch.tensor(token_type_ids)
               #           }
  

               print("START EMBEDDINGS")
               # Perform the forward pass WITH gradients DISABLED for efficiency
               with torch.no_grad(): 
                    embeddings = model_passages_encoder(**inputs).pooler_output   
                    # TO SHOW THE "DIMENSION" OF THE EMBEDDINGS VECTORS IMPLEMENTED BY "THIS" MODEL: "DPRContextEncoder.from_pretrained(trained_model_used_in_context_encoder)"
                    # FROM EXPERIENCE USING THIS MODEL THE VECTOR HAS DIMENSION 768. WILL BE CONFIRMED FURTHER DOWN
                    # print(embeddings.shape)

               # Convert to NumPy AFTER Detaching FROM GPU COMPUTATIONAL GRAPH AND MOVING to CPU 
               # Some models (especially those optimized for GPUs) use float16 for performance BUT OTHER LIBRARIES expect float32 (e.g., FAISS, Scikit-learn, ANN)
               # FOR FURTHER PROCESSING THE OUTPUT. THUS, EXPLICITLY "CASTING" prevents issues 
               # embeddings_numpy = embeddings.detach().cpu().numpy().astype("float32")
               embeddings_numpy = embeddings.detach().cpu().numpy()    
                   
               print("SHAPE FOR CHUNK OF EMBEDDED DOCS:", embeddings_numpy.shape)    

               batch_of_embeddings.append(embeddings_numpy)
               print("EMBEDDINGS FINISHED")               
               print(f"LENGTH OF batch_of_embeddings {len(batch_of_embeddings)}")    

               if len(batch_of_embeddings)>batch_size:
                    
                    print(len(batch_of_embeddings))                                  

                    # "h5py.File(output_path, 'a')" OPENS A IN  "append" mode WITHOUT DELETING EXISTING data OR AUTOMATICALLY CREATE IT "at output_path"
                    # IF IT DOES NOT ALREADY EXIST.                    
                    with h5py.File(output_path, 'a') as f2:

                         # AN HD5 (Hierarchical Data Format version 5) FILE CAN CONTAIN MULTIPLE DATASETS. WE CALL "dataset" THE ONE WE ARE CREATING
                         # Check if the dataset already exists OTHERWISE creates it with expandable dimensions
                         if "dataset" not in f2:

                              # WHEN INITIALIZING "dataset" ALLOW UNLIMITED ROWS VIA "None" (HERE EACH ROW WILL BE A CHUNK). "dataset" WILL INCREMENTALLY 
                              # RECEIVE EMBEDDINGS OF DOCUMENTS IN CHUNKS OF NUMPY ARRAYS HAVING EACH THE NUMBER OF "COLUMNS" DEFINED BY: 
                              # batch_of_embeddings[0].shape[1:] WHERE "batch_of_embeddings[0]" IS THE FIRST ARRAY IN THE LIST "batch_of_embeddings" AND
                              # ".shape[1:]" IS THE SHAPE OF THE THAT ARRAY EXCEPT THE FIRST DIMENSION. THUS IT IS JUST THE NUMBER OF "COLUMNS"
                              # NOTE: WE SKIPPED GRABBING THE FIRST DIMENSION (# OF ROWS) BECAUSE WE ALREADY ADDRESSED THAT VIA "NONE"
                                                          
                              maxshape = (None,) + batch_of_embeddings[0].shape[1:]
                              # CREATE WITH 0 ROWS AND "batch_of_embeddings[0].shape[1:]" NUMBER OF COLUMNS
                              dset = f2.create_dataset("dataset", shape=(0,) + batch_of_embeddings[0].shape[1:], 
                                                      maxshape=maxshape, 
                                                      # SPECIFY THAT EACH CHUNK APPENDED MUST HAVE AS NUMBER OF ROWS: "batch_of_embeddings[0].shape[0]"
                                                      # CAN BE ANY NUMBER OF YOUR CHOICE. THE SYSTEM INTERNALLY SPLITS THE NUMBER OF ROW RECEIVED TO CREATE
                                                      # THE SPECIFY NUMBER OF ROWS IN EACH CHUNK WITH The HDF5 format ALLOWING partial chunk UNITL MORE DATA
                                                      # COMES IN LATER
                                                      # chunks=((batch_of_embeddings[0].shape[0]),) + \
                                                      chunks=((batch_of_embeddings[0].shape[0])/chunks_grabbed_from_source_h5_file,) + \
                                                     
                                                                                                              batch_of_embeddings[0].shape[1:])

                         else:
                              # Reopen the existing dataset if it already exists
                              dset = f2["dataset"]


                         # COULD DO USE "np.stack([np_arr for np_arr in batch_of_embeddings])" HERE INSTEAD OF THE LOOP BUT NOT SURE MEMORY CAN HANDLE IT!!!!
                         # HOWEVER IT WOULD DO BETTER BECAUSE ONLY A SINGLE "WRITE" OPERATION BUT WE LOOSE THE FLEXIBILITY OF ADDING "SMALLER" CHUNKS IN LATER
                         # RUNS. np.vstack() JUST CONCATENATE ALONG axis 0 (all arrays MUST HAVE the same shape except in this dimension) 
                         # WHILE np.stack() ALSO ADDS A NEW DIMENSION TO THE OUTPUT: INPUT ARRAYS OF (100, 50) GIVE 10, 100, 50).
                         # This WOULD BE useful if you want to keep each INPUT ARRAY AS separate SLICE in the stacked result 
                         # IT WOULD WORK AS FOLLOWS:
                         # stacked_array = np.vstack(batch_of_embeddings)
                         # Resize dataset to accommodate the stacked data
                         #dset.resize((dset.shape[0] + stacked_array.shape[0]), axis=0)
                         # Write the entire stacked array to the end of the dataset
                         #dset[-stacked_array.shape[0]:] = stacked_array
                         for array in batch_of_embeddings:

                              # Resize dataset to accommodate new data
                              dset.resize((dset.shape[0] + array.shape[0]), axis=0)
                              
                              # Write data to the end of the dataset
                              dset[-array.shape[0]:] = array

                    # Clear the batch and row numbers after writing to the file  
                    del embeddings                  
                    del embeddings_numpy
                    batch_of_embeddings.clear()     
                    gc.collect()          
       
    if len(batch_of_embeddings)>0: 
                              
          print(len(batch_of_embeddings))          

          with h5py.File(output_path, 'a') as f2:

               # AN HD5 (Hierarchical Data Format version 5) FILE CAN CONTAIN MULTIPLE DATASETS. WE CALL "dataset" THE ONE WE ARE CREATING
               # Check if the dataset already exists OTHERWISE creates it with expandable dimensions
               if "dataset" not in f2:

                    # WHEN INITIALIZING "dataset" ALLOW UNLIMITED ROWS VIA "None" (HERE EACH ROW WILL BE A CHUNK). "dataset" WILL INCREMENTALLY 
                    # RECEIVE EMBEDDINGS OF DOCUMENTS IN CHUNKS OF NUMPY ARRAYS HAVING EACH THE NUMBER OF "COLUMNS" DEFINED BY: 
                    # batch_of_embeddings[0].shape[1:] WHERE "batch_of_embeddings[0]" IS THE FIRST ARRAY IN THE LIST "batch_of_embeddings" AND
                    # ".shape[1:]" IS THE SHAPE OF THE THAT ARRAY EXCEPT THE FIRST DIMENSION. THUS IT IS JUST THE NUMBER OF "COLUMNS"
                    # NOTE: WE SKIPPED GRABBING THE FIRST DIMENSION (# OF ROWS) BECAUSE WE ALREADY ADDRESSED THAT VIA "NONE"
                                             
                    maxshape = (None,) + batch_of_embeddings[0].shape[1:]
                    # CREATE WITH 0 ROWS AND "batch_of_embeddings[0].shape[1:]" NUMBER OF COLUMNS
                    dset = f2.create_dataset("dataset", shape=(0,) + batch_of_embeddings[0].shape[1:], 
                                        maxshape=maxshape, 
                                        # SPECIFY THAT EACH CHUNK APPENDED MUST HAVE AS NUMBER OF ROWS: "batch_of_embeddings[0].shape[0]"
                                        # CAN BE ANY NUMBER OF YOUR CHOICE. THE SYSTEM INTERNALLY SPLITS THE NUMBER OF ROW RECEIVED TO CREATE
                                        # THE SPECIFY NUMBER OF ROWS IN EACH CHUNK WITH The HDF5 format ALLOWING partial chunk UNITL MORE DATA
                                        # EVENTUALLY COMES IN LATER
                                        # chunks=((batch_of_embeddings[0].shape[0]),) + \
                                        chunks=((batch_of_embeddings[0].shape[0])/chunks_grabbed_from_source_h5_file,) + \
                                        
                                                                                                              batch_of_embeddings[0].shape[1:])
                    

               else:
                    # # Reopen the existing dataset if it already exists
                    dset  = f2["dataset"]


               for ind, array in enumerate(batch_of_embeddings):

                    print("lAST BATCH: LIST ITEM #:", ind)
                    # Resize dataset to accommodate new data
                    dset.resize((dset.shape[0] + array.shape[0]), axis=0)
                    
                    # Write data to the end of the dataset
                    dset[-array.shape[0]:] = array

          # Clear the batch and row numbers after writing to the file  
          del embeddings        
          del embeddings_numpy
          batch_of_embeddings.clear() 
          gc.collect() 
                    

In [ ]:
%%time


# NUMBER OF CHUNKS IN SOURCE H5 FILE GRABBED AT ONCE. MUST BE A "DIVISOR" OF data_size/divisor_value:  
# [1, 2, 3, 4, 6, ....
chunks_grabbed_from_source_h5_file =2
batch_size=3
output_path_for_text_passage_embeddings = os.path.join(datasets_storage_on_disk, "data","text_passage_embeddings.h5")



create_text_passage_embeddings(file_source_path=path_to_inputs_tensor_as_numpy,
                               chunks_grabbed_from_source_h5_file=chunks_grabbed_from_source_h5_file,
                               batch_size=batch_size,
                               output_path=output_path_for_text_passage_embeddings,
                               )

In [ ]:


with h5py.File(output_path_for_text_passage_embeddings, 'r') as f:

    for key in f.keys():

        dataset = f[key]
        print(f"{key} : {dataset.shape}")

dataset : (1755520, 768)


### <a id='toc2_1_4_'></a>[III- 6. CREATE FAISS INDECES FROM TEXT PASSAGES EMBEDDINGS](#toc0_)
 

In [11]:
#device = torch.device('cuda:0')

def create_faiss_IndexIVFFlat_index(file_path_for_embeddings_hd5, indeces_path, chunk_size, divisor, number_cluster_for_embeddings=100, 
                                    sample_embeddings_size=50000):

    # Open the HDF5 file
    with h5py.File(file_path_for_embeddings_hd5, 'r') as f: 

    # Iterate over the keys associated with the different datasets inside the HD5 file
     for key in f.keys():  

        # Grabs iteratively each dataset 
        dataset = f[key]

        # Dimensionality of embeddings
        embeddings_vectors_length = dataset.shape[1]

        # NUMBER OF VECTORS EMBEDDINGS (ROWS)
        total_embeddings = dataset.shape[0]
        
        # IF EMBEDDINGS ARE "NORMALIZED":
        # Initialize a quantizer
        # quantizer = faiss.IndexFlatIP(d) 
        # Initialize the IVF index
        # index_cpu = faiss.IndexIVFFlat(quantizer, d, nlist, faiss.METRIC_COSINE)  
              
        quantizer = faiss.IndexFlatL2(embeddings_vectors_length) 
        # Initialize the IVF index
        # "number_cluster_for_embeddings" IS THE Number of clusters INTO WHICH that ALL THE EMBEDDINGS FROM THE SOURCE FILE will be partitioned.
        index_cpu = faiss.IndexIVFFlat(quantizer, embeddings_vectors_length, number_cluster_for_embeddings, faiss.METRIC_L2)

        # "index.nprobe" IS typically set after the index has been created and before performing a search
        # "index.nprobe" SPECIFIES HOW MANY OF THE CLUSTERS MADE USING THE EMBEDDINGS WILL BE INCLUDED FOR A SEARCH, balancing EFFICIENCY and ACCUARCY.                 
        # BY Setting nprobe equal to nlist (number_cluster_for_embeddings) the index will search "ALL" clusters in the index TO MAXIMIZE "RECALL".
        index_cpu.nprobe = number_cluster_for_embeddings

        # Transfer CPU index to GPU (device 0)
        try:            
            #print("FAISS-GPU resources are available.")
            gpu_resources = faiss.StandardGpuResources()
            index = faiss.index_cpu_to_gpu(gpu_resources, 0, index_cpu)  
        except AttributeError:
            print(f" FAISS-GPU resources are not available. Ensure the GPU version of FAISS is installed.", "\n")
        
        
        # SPECIFY THE SIZE OF EMBEDDINGS SAMPLE TO BE USED FOR "TRAINING" the index
        sample_embeddings = dataset[:sample_embeddings_size]          
        index.train(sample_embeddings)

        # TRANSFER BACK THE "TRAINED" INDEX TO CPU BEFORE SAVINGS
        index_cpu = faiss.index_gpu_to_cpu(index)

        # Set up index to use disk storage
        # If the index size exceeds available MEMORY, FAISS AUTOMATICALLY WILL USE disk-backed storage.        
        faiss.write_index(index_cpu, indeces_path)

        # SETS TO USE GPU FOR INDEXING THE EMBEDDINGS "AFTER" THE INDEX HAS BEEN TRAINED
        index = faiss.index_cpu_to_gpu(gpu_resources, 0, index_cpu)

        # Add embeddings in chunks TO THE INDEX
        for start in range(0, total_embeddings, chunk_size):
            end = min(start + chunk_size, total_embeddings)
            embeddings_chunk = np.array(dataset[start:end], dtype='float32')
            # Add to disk-backed index
            index.add(embeddings_chunk)  

            # Periodically save to disk
            
            if start % (chunk_size * divisor) == 0: 

                index_cpu = faiss.index_gpu_to_cpu(index) 
                # USE "partial_index" SCHEMES AVOIDS fragmentation AND unnecessary I/O on THE "FINAL" file
                faiss.write_index(index_cpu, os.path.join(datasets_storage_on_disk,"data","partial_index.faiss"))
                print(f"Partial Index saved at chunk starting at row: {start} and ending at row: {end}")
                index = faiss.index_cpu_to_gpu(gpu_resources, 0, index_cpu)

        index_cpu = faiss.index_gpu_to_cpu(index) 
        
        # Final save to ensure all embeddings are written
        faiss.write_index(index_cpu, indeces_path)

In [12]:
%%time

#output_path_for_text_passage_embeddings = os.path.join(datasets_storage_on_disk, "data","text_passage_embeddings.h5")

path_to_save_faiss_indeces= os.path.join(datasets_storage_on_disk, "data","disk_index.faiss")
chunks_of_embeddings = int(data_size/4)
divisor = 26
number_cluster_for_embeddings = 2048 #Max possible!!!!)
sample_embeddings_size = data_size

create_faiss_IndexIVFFlat_index(file_path_for_embeddings_hd5=output_path_for_text_passage_embeddings, 
                                indeces_path=path_to_save_faiss_indeces, 
                                chunk_size=chunks_of_embeddings,
                                divisor = divisor,
                                number_cluster_for_embeddings=number_cluster_for_embeddings, 
                                sample_embeddings_size=sample_embeddings_size)

Partial Index saved at chunk starting at row: 0 and ending at row: 1755520
CPU times: user 1min 57s, sys: 12.6 s, total: 2min 9s
Wall time: 2min 43s


In [ ]:
%%time


vectors_to_show = 1

# TO "USE" A FAISS INDEX ALREADY "STORED" ON DISK WITHOUT LOADING IT "ENTIRELY" INTO MEMORY, USE "MEMORY-MAPPED" DISK STORAGE MODE.
# TO ONLY INSPECT THE "METADATA" WITHOUT LOADING THE "FULL" INDEX USE THE ".read_index()" METHOD THE WITH "faiss.IO_FLAG_MMAP" PARAMETER
print(f"\nLoading FAISS index from: {path_to_save_faiss_indeces} (Memory-Mapped Mode)")
index = faiss.read_index(path_to_save_faiss_indeces, faiss.IO_FLAG_MMAP)
print("\nFAISS index successfully loaded in memory-mapped mode ")

# Get the number of vectors stored in the index
num_vectors = index.ntotal  

print(f"\nNumber of vectors in the index: {num_vectors}\n")

# Extract the first "vectors_to_show" vectors (if available)
if num_vectors >= vectors_to_show:
    vectors = np.zeros((vectors_to_show, index.d), dtype=np.float32)  # Create an empty NumPy array
    
    # Retrieve first 5 vectors directly
    vectors = index.reconstruct_n(0, vectors_to_show)
    
    print(f"First {vectors_to_show} vectors in FAISS Index:")
    print(vectors)
else:
    print(f"The FAISS index has less than {vectors_to_show} vectors.")


#del index


Loading FAISS index from: /mnt/f/RAG DATASETS/data/disk_index.faiss (Memory-Mapped Mode)

FAISS index successfully loaded in memory-mapped mode 

Number of vectors in the index: 1755520

First 1 vectors in FAISS Index:
[[ 6.26053572e-01  1.80991575e-01  1.84850961e-01 -3.43627572e-01
  -1.00219652e-01  3.74523997e-01 -1.18327737e-01  4.07786965e-01
   7.28981197e-02 -1.73243165e-01  4.62035723e-02  5.30305684e-01
  -2.22905695e-01  7.20958635e-02 -2.12288484e-01 -2.46788859e-01
   6.29681274e-02  3.51284862e-01 -1.17545463e-01 -1.56658351e-01
  -1.31592703e+00 -4.11132090e-02 -2.58875996e-01 -1.81244135e-01
   8.30460072e-01  1.45105049e-01  3.53189647e-01 -3.43689889e-01
  -5.64491034e-01 -5.69420122e-02  7.03412235e-01  3.21658522e-01
   2.97736347e-01  5.08594476e-02 -4.72325146e-01 -1.67698249e-01
  -6.87645823e-02 -2.56237239e-01 -4.69520211e-01 -5.22452295e-01
  -6.75320745e-01  2.34252170e-01  2.09486082e-01  2.46723413e-01
  -2.91404128e-01 -5.81698418e-01 -1.86656082e+00  4.5

### <a id='toc2_1_5_'></a>[III- 7. EXAMPLE: FINDING "BY HAND" SIMILAR DOCUMENTS TO A QUESTION](#toc0_)
 

#### <a id='toc2_1_5_1_'></a>[III- 7.1 EMBED THE USER "PROMPT/QUESTION" INTO A VECTOR](#toc0_)

In [16]:

def embed_question_as_vector(question):

    device = torch.device('cuda:0')

    # INSTANTIATE THE pre-trained TOKENIZER TO BE APPLIED THE TEXT PASSAGES (AKA KNOWLEDGE BASE )
    trained_model_used_in_question_tokenizer = 'facebook/dpr-question_encoder-single-nq-base'

    # tokenizer_passage_encoder = AutoTokenizer.from_pretrained(trained_model_used_in_context_tokenizer) 
    # tokenizer_passage_encoder = DPRQuestionEncoderTokenizer.from_pretrained(trained_model_used_in_context_tokenizer)     
    # REGARDING WARNING AT EXECUTION TIME: "The tokenizer class you load from this checkpoint is not the same type as the class this function is called from"
    # Functionally, both DPRQuestionEncoderTokenizer and DPRContextEncoderTokenizer tokenizers are identical, so it doesn’t impact model performance. 
    tokenizer_question_encoder = DPRQuestionEncoderTokenizer.from_pretrained(trained_model_used_in_question_tokenizer) 


    
    #inputs = tokenizer_question_encoder(question, return_tensors='pt', padding=True, max_length=max_length, truncation=True).to(device)
    inputs = tokenizer_question_encoder(question, return_tensors='pt').to(device)


    model_question_encoder = DPRQuestionEncoder.from_pretrained(trained_model_used_in_question_tokenizer) 
    model_question_encoder = model_question_encoder.to(device)
    # Perform the forward pass WITH gradients DISABLED for efficiency
    with torch.no_grad():
        question_embedding = model_question_encoder(**inputs).pooler_output

    embedding_numpy = question_embedding.cpu().detach().numpy().astype('float32')
    print("\n", "Shape of Embedded Question", embedding_numpy.shape)

    return embedding_numpy



In [ ]:
#user_question =  "Who won the world series in 2020?"
user_question= "Human fertilization"

question_embedding_numpy = embed_question_as_vector(user_question)    


Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.bias', 'question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



 Shape of Embedded Question (1, 768)


#### <a id='toc2_1_5_2_'></a>[III- 7.2 RETRIEVE RELEVANT DOCUMENTS FOR THE USER "PROMPT/QUESTION"](#toc0_)
 

In [18]:
def retrieve_documents_similar_to_question(path, embedded_question, number_docs):
        
    # TO "USE" A FAISS INDEX ALREADY "STORED" ON DISK WITHOUT LOADING IT "ENTIRELY" INTO MEMORY, USE "MEMORY-MAPPED" DISK STORAGE MODE.
    # TO ONLY INSPECT THE "METADATA" WITHOUT LOADING THE "FULL" INDEX USE THE ".read_index()" METHOD THE WITH "faiss.IO_FLAG_MMAP" PARAMETER
    print(f"\nLoading FAISS index from: {path} (Memory-Mapped Mode)")
    index = faiss.read_index(path, faiss.IO_FLAG_MMAP)
    print("\nFAISS index successfully loaded in memory-mapped mode ")


    # Convert the index to a GPU index
    gpu_resources = faiss.StandardGpuResources()
    gpu_index = faiss.index_cpu_to_gpu(gpu_resources, 0, index)  # 0 is the GPU ID


    
    # Perform the search on GPU
    D, I = gpu_index.search(embedded_question, k=number_docs)

    print("\n", "Distances to Question:", D, "\n", "FAISS Indeces of Relevant Context Documents:", I)

    return D, I


In [ ]:
number_of_top_passages_to_keep = 4

D, I = retrieve_documents_similar_to_question(path_to_save_faiss_indeces, question_embedding_numpy, number_of_top_passages_to_keep)


Loading FAISS index from: /mnt/f/RAG DATASETS/data/disk_index.faiss (Memory-Mapped Mode)

FAISS index successfully loaded in memory-mapped mode 

 Distances to Question: [[74.96144  75.66108  77.482864 77.482864]] 
 FAISS Indeces of Relevant Context Documents: [[791664  42048 732162 909755]]


#### <a id='toc2_1_5_3_'></a>[III- 7.3 CHECK THE RELEVANT DOCUMENTS RETRIEVED FOR THE USER "PROMPT/QUESTION"](#toc0_)
 

In [21]:

def map_documents_to_faiss_id(tsv_file, my_list):
    """
    Reads only the rows specified in `my_list` from a large TSV file.
    
    :param tsv_file: Path to the TSV file.
    :param my_list: List of row numbers to extract (1-based index).
    :return: List of extracted rows.
    """
    extracted_documents = []

    with open(tsv_file, "r", encoding="utf-8") as file:
        
        for row_number, line in enumerate(file, start=0):  # FIRST ROW IS AT INDEX 0 AND THERE IS NO HEADER
            if row_number in my_list:
                columns = line.strip().split("\t")  # Split by tab to get columns. GET THE LINE THAN SPLIT ON ITS COLUMNS
                extracted_documents.append((row_number,columns[1]))  # Extract the second column (index 1)
                
            if row_number > max(my_list):  # Stop early if max row is reached
                break

    return extracted_documents


In [23]:

# Example usage

# "I" IS A LIST OF ONLY 1 ELEMENT. THAT ELEMENT IS A 1-D NUMPY ARRAY
row_numbers_of_relevant_documents_list =  list(I[0]) # Row indices (0-based)


path_to_passages_that_were_embedded = tsv_file_path1
print(path_to_passages_that_were_embedded)

relevant_documents_list = map_documents_to_faiss_id(path_to_passages_that_were_embedded, row_numbers_of_relevant_documents_list)

len(relevant_documents_list)


/mnt/f/RAG DATASETS/data/text_passages_reduced_length.tsv


4

In [ ]:

print(f"\nTop {number_of_top_passages_to_keep} Relevant Contexts:")

for i, doc in enumerate(relevant_documents_list):

    distance_index = row_numbers_of_relevant_documents_list.index(doc[0])
    print(f"\n", f"{i+1}: {doc[1]}", "\n", \
           f"Distance {D[0][distance_index]} (Passage ID: {doc[0]})")



Top 4 Relevant Contexts:

 1: genetic controls of female development, and much remains unknown about the female embryonic process. males become externally distinct between 8 and 12 weeks, as androgens enlarge the phallus and cause the urogenital groove and sinus to fuse in the midline, producing an unambiguous penis with a phallic urethra, and a thinned, rugated scrotum. dihydrotestosterone will differentiate the remaining male characteristics of the external genitalia. a sufficient amount of any androgen can cause external masculinization. the most potent is dihydrotestosterone ( dht ), generated from testosterone in skin and genital tissue by the action of 5 \ u03b1 - reductase. a male fetus may be incompletely masculinized if this enzyme is deficient. in some diseases and circumstances, other androgens may be present in high enough concentrations to cause partial or ( rarely ) complete masculinization of the external genitalia of a genetically female fetus. the testes begin to secr

#### <a id='toc2_1_5_4_'></a>[III- 7.4 COMPARE BART LLM RESPONSE WITH AND WIHTOUT CONTEXT](#toc0_)
 

In [25]:
def generate_answer_without_rerieval(question):
    
    # News summarization
    #pre_trained_model = "facebook/bart-large-cnn"
    # Lighter version for inference
    #pre_trained_model = "facebook/bart-base"
    # General text generation
    pre_trained_model = "facebook/bart-large"


    # Hugging Face's TOKENIZERS "CANNOT" USE GPU Because Tokenizations are purely CPU-based operations.
    tokenizer = BartTokenizer.from_pretrained(pre_trained_model)
    inputs = tokenizer(question, return_tensors = 'pt', max_length=1024, padding=True, truncation = True)
    
    device = torch.device('cuda:0')
    model_for_tokenizer = BartForConditionalGeneration.from_pretrained(pre_trained_model).to(device)

    with torch.no_grad(): 
        response_ids = model_for_tokenizer.generate(inputs['input_ids'].to(device), max_length=256, min_length=40, length_penalty = 2.0, \
                                                    num_beams = 4, early_stopping = True)
    
    #print("\nresponse_ids:\n", response_ids)
    answer = tokenizer.decode(response_ids[0].detach().cpu(), skip_special_tokens = True)

    print(f"\nGenerated Answer Without Rerieval: \n {answer}")

    return answer

In [26]:
#user_question =  "Who won the world series in 2020?"
user_question= "Human fertilization"

generate_answer_without_rerieval(user_question)


Generated Answer Without Rerieval: 
 Human fertilizationFertilizationFetilisationFetal implantationFetabolismFetanomycinFetanylphosphataseFetylphosphateFetanolophosphate


'Human fertilizationFertilizationFetilisationFetal implantationFetabolismFetanomycinFetanylphosphataseFetylphosphateFetanolophosphate'

In [27]:
def generate_answer_using_rerieval(context):
    
    # News summarization
    #pre_trained_model = "facebook/bart-large-cnn"
    # Lighter version for inference
    #pre_trained_model = "facebook/bart-base"
    # General text generation
    pre_trained_model = "facebook/bart-large"


    # Hugging Face's TOKENIZERS "CANNOT" USE GPU Because Tokenizations are purely CPU-based operations.
    tokenizer = BartTokenizer.from_pretrained(pre_trained_model)

    input_text = " ".join(context)
    inputs = tokenizer(input_text, return_tensors = 'pt', max_length=1024, padding=True, truncation = True)
    
    device = torch.device('cuda:0')
    model_for_tokenizer = BartForConditionalGeneration.from_pretrained(pre_trained_model).to(device)

    with torch.no_grad(): 
        response_ids = model_for_tokenizer.generate(inputs['input_ids'].to(device), max_length=256, min_length=40, length_penalty = 2.0, \
                                                    num_beams = 4, early_stopping = True)
        
    answer = tokenizer.decode(response_ids[0].detach().cpu(), skip_special_tokens = True)
    
    #print("\nresponse_ids:\n", response_ids)
    print(f"\nGenerated Answer USING Rerieval: \n {answer}")

    return answer

In [ ]:
top_contexts = [doc[1] for doc in relevant_documents_list]
generate_answer_using_rerieval(top_contexts)


Generated Answer USING Rerieval: 
 genetic controls of female development, and much remains unknown about the female embryonic process. in most multicellular organisms, the mtdna - - or mitogenome - - is organized as a circular, covalently closed, double - stranded dna. but in many unicellular ( e. g. the ciliate tetrahymena or the green alga chlamydomonas reinhardtii ) fertilization while the remains of the dominant follicle in the ovary become a corpus luteum ; this body has a primary function of producing large amounts of progesterone. the hormone drop causes the uterus to shed its lining in a process termed menstruation. menstruation also occurs in closely related primates ( apes and monkeys ). the average age of menarche is 12 - - 15. they may occasionally start as early as eight, and this onset may still be normal. the cessation of menstrual cycles at the end of a woman ' s human male. this first period often occurs later in the developing world than the developed world. factors

"genetic controls of female development, and much remains unknown about the female embryonic process. in most multicellular organisms, the mtdna - - or mitogenome - - is organized as a circular, covalently closed, double - stranded dna. but in many unicellular ( e. g. the ciliate tetrahymena or the green alga chlamydomonas reinhardtii ) fertilization while the remains of the dominant follicle in the ovary become a corpus luteum ; this body has a primary function of producing large amounts of progesterone. the hormone drop causes the uterus to shed its lining in a process termed menstruation. menstruation also occurs in closely related primates ( apes and monkeys ). the average age of menarche is 12 - - 15. they may occasionally start as early as eight, and this onset may still be normal. the cessation of menstrual cycles at the end of a woman ' s human male. this first period often occurs later in the developing world than the developed world. factors such as genetics, diet and overall

### <a id='toc2_1_6_'></a>[IV- ORGANIZE TEXT PASSAGES AND THEIR EMBEDDINGS IN THE FORMAT NEEDED BY THE RAG "MODEL"](#toc0_)

#### <a id='toc2_1_6_1_'></a>[IV- 11.1 JOIN TEXT PASSAGES AND THEIR EMBEDDINGS IN THE SAME FILE](#toc0_)

In [29]:
%%time

# FIND MAX LENGHTS ACCROSS ALL TEXT PASSAGES


# RECALL: tsv_file_path1 = os.path.join(datasets_storage_on_disk, "data","text_passages_reduced_length.tsv")

#for chunk_df in pd.read_csv(file_path, skiprows=range(1, start_row + 1), chunksize=chunk_size):
for chunk in pd.read_csv(tsv_file_path1, sep='\t', names=['id', 'text'], chunksize=50000):  

    passages_max_len = 0
    column_name = chunk.columns[1]
    length_of_texts = [len(row[column_name]) for index, row in chunk.iterrows()]
    if passages_max_len < max(length_of_texts):
        passages_max_len =  max(length_of_texts)

passages_max_len = passages_max_len +10
print(f"Maximum Length of Passages: {passages_max_len}")

Maximum Length of Passages: 1629
CPU times: user 3min 48s, sys: 1.09 s, total: 3min 49s
Wall time: 4min 20s


In [30]:
def create_file_containing_text_passages_with_embeddings(text_passages_path, file_path_for_embeddings_hd5, 
                                                           chunk_size, batch_size, max_len, output_path, index_size):


    # Open the HDF5 file
    with h5py.File(file_path_for_embeddings_hd5, 'r') as f: 
     

    # Iterate over the keys associated with the different datasets inside the HD5 file
     for key in f.keys():  

        # Grabs iteratively each dataset 
        dataset = f[key]
        total_embeddings = dataset.shape[0]
        #print(total_embeddings)

        batch_of_dicts = []

        total_rows_loaded = 0

        #for chunk_df in pd.read_csv(file_path, skiprows=range(1, start_row + 1), chunksize=chunk_size):
        for chunk in pd.read_csv(text_passages_path, sep='\t', names=['id', 'text'], chunksize=chunk_size): 


            if total_rows_loaded == index_size:
                print(f"Loaded ALREADY exactly {total_rows_loaded} rows. Stopping.")
                break  # Stop the current "FOR" loop AND PROCESS THE CODE AFTER THE IT. "return" WOULD Completely exit the function!
            
            column_name = chunk.columns[1]
            
            list_of_dicts_from_chunks = [{column_name: row[column_name], "title": 'passage '+ str(index), "embeddings": dataset[index]} 
                                                                                                       for index, row in chunk.iterrows()
                                        ] 
            
            batch_of_dicts = batch_of_dicts + list_of_dicts_from_chunks
            #print(len(batch_of_dicts))         
              

            if len(batch_of_dicts)> batch_size:
                        
                print(len(batch_of_dicts)) 

                # 'S100' is for a fixed-length ASCII bytes USED FOR STRINGS (HERE 100 BYTES)
                # ('f4', (chunk_size, 768)) is for  N-D numpy arrays with shape (chunk_size, 768) of float32 ('f4') type
                vector_sizes = batch_of_dicts[0]["embeddings"].shape[0]
                #print(vector_sizes)

                dtype = np.dtype([(column_name, 'S' + str(max_len)),  ('title', 'S' + str(30)),  
                                                    ('embeddings', ('f4', (1, vector_sizes))) ])

                #structured_array = np.array([tuple(d.values()) for d in batch_of_dicts], dtype=dtype)
                structured_array = np.array([(d[column_name].encode('utf-8'), d["title"].encode('utf-8'), d["embeddings"]) 
                                                                                    for d in batch_of_dicts], dtype=dtype)
                             

                # "h5py.File(output_path, 'a')" OPENS A IN  "append" mode WITHOUT DELETING EXISTING data OR AUTOMATICALLY CREATE IT "at output_path"
                # IF IT DOES NOT ALREADY EXIST.                    
                with h5py.File(output_path, 'a') as f2:

                        # AN HD5 (Hierarchical Data Format version 5) FILE CAN CONTAIN MULTIPLE DATASETS. WE CALL "dataset" THE ONE WE ARE CREATING
                        # Check if the dataset already exists OTHERWISE creates it with expandable dimensions
                        if "dataset" not in f2:

                            # WHEN INITIALIZING "dataset" ALLOW UNLIMITED ROWS VIA "None" (HERE EACH ROW WILL BE A CHUNK). "dataset" WILL INCREMENTALLY 
                            # RECEIVE EMBEDDINGS OF DOCUMENTS IN CHUNKS OF NUMPY ARRAYS HAVING EACH THE NUMBER OF "COLUMNS" DEFINED BY: 
                            # batch_of_embeddings[0].shape[1:] WHERE "batch_of_embeddings[0]" IS THE FIRST ARRAY IN THE LIST "batch_of_embeddings" AND
                            # ".shape[1:]" IS THE SHAPE OF THE THAT ARRAY EXCEPT THE FIRST DIMENSION. THUS IT IS JUST THE NUMBER OF "COLUMNS"
                            # NOTE: WE SKIPPED GRABBING THE FIRST DIMENSION (# OF ROWS) BECAUSE WE ALREADY ADDRESSED THAT VIA "NONE"
                                                        
                            maxshape = (None,) + structured_array.shape[1:]
                            
                            # CREATE WITH 0 ROWS AND "batch_of_embeddings[0].shape[1:]" NUMBER OF COLUMNS
                            dset = f2.create_dataset("dataset", data=structured_array, maxshape=maxshape)

                            # Add column names as metadata
                            dset.attrs['column_names'] = [column_name, 'title', 'embeddings']
                                                  
                        else:
                            # Reopen the existing dataset if it already exists
                            dset = f2["dataset"]
                            dset.resize((dset.shape[0] + len(batch_of_dicts)), axis=0)
                            dset[-len(batch_of_dicts):] = structured_array


                # Clear the batch and row numbers after writing to the file         
                list_of_dicts_from_chunks.clear()
                batch_of_dicts.clear()    
                del structured_array 
                gc.collect()   

            total_rows_loaded += len(chunk)

    if len(batch_of_dicts)>0: 
                            
        print(len(batch_of_dicts))       

        # 'S100' is for a fixed-length ASCII bytes USED FOR STRINGS (HERE 100 BYTES)
        # ('f4', (chunk_size, 768)) is for  N-D numpy arrays with shape (chunk_size, 768) of float32 ('f4') type
        vector_sizes = batch_of_dicts[0]["embeddings"].shape[0]
        #print(vector_sizes)

        dtype = np.dtype([(column_name, 'S' + str(max_len)),  ('title', 'S' + str(30)),  ('embeddings', ('f4', (1, vector_sizes))) ])
        
        #structured_array = np.array([tuple(d.values()) for d in batch_of_dicts], dtype=dtype)
        structured_array = np.array([(d[column_name].encode('utf-8'), d["title"].encode('utf-8'), d["embeddings"]) 
                                                                                                                      for d in batch_of_dicts], dtype=dtype)

        # "h5py.File(output_path, 'a')" OPENS A IN  "append" mode WITHOUT DELETING EXISTING data OR AUTOMATICALLY CREATE IT "at output_path"
        # IF IT DOES NOT ALREADY EXIST.                    
        with h5py.File(output_path, 'a') as f2:

                # AN HD5 (Hierarchical Data Format version 5) FILE CAN CONTAIN MULTIPLE DATASETS. WE CALL "dataset" THE ONE WE ARE CREATING
                # Check if the dataset already exists OTHERWISE creates it with expandable dimensions
                if "dataset" not in f2:

                    # WHEN INITIALIZING "dataset" ALLOW UNLIMITED ROWS VIA "None" (HERE EACH ROW WILL BE A CHUNK). "dataset" WILL INCREMENTALLY 
                    # RECEIVE EMBEDDINGS OF DOCUMENTS IN CHUNKS OF NUMPY ARRAYS HAVING EACH THE NUMBER OF "COLUMNS" DEFINED BY: 
                    # batch_of_embeddings[0].shape[1:] WHERE "batch_of_embeddings[0]" IS THE FIRST ARRAY IN THE LIST "batch_of_embeddings" AND
                    # ".shape[1:]" IS THE SHAPE OF THE THAT ARRAY EXCEPT THE FIRST DIMENSION. THUS IT IS JUST THE NUMBER OF "COLUMNS"
                    # NOTE: WE SKIPPED GRABBING THE FIRST DIMENSION (# OF ROWS) BECAUSE WE ALREADY ADDRESSED THAT VIA "NONE"
                                                
                    maxshape = (None,) + structured_array.shape[1:]
                    
                    # CREATE WITH 0 ROWS AND "batch_of_embeddings[0].shape[1:]" NUMBER OF COLUMNS
                    dset = f2.create_dataset("dataset", data=structured_array, maxshape=maxshape)

                    # Add column names as metadata
                    dset.attrs['column_names'] = [column_name, 'title', 'embeddings']
                                            
                else:
                    # Reopen the existing dataset if it already exists
                    dset = f2["dataset"]
                    dset.resize((dset.shape[0] + len(batch_of_dicts)), axis=0)
                    dset[-len(batch_of_dicts):] = structured_array


        # Clear the batch and row numbers after writing to the file         
        list_of_dicts_from_chunks.clear()
        batch_of_dicts.clear()    
        del structured_array 
        gc.collect()          

In [39]:
%%time

output_path_for_text_passage_embeddings = os.path.join(datasets_storage_on_disk, "data","text_passage_embeddings.h5")
chunk_size_for_text_passages_with_embeddings = 109720
batch_size_for_text_passages_with_embeddings = chunk_size_for_text_passages_with_embeddings*4

output_path_for_text_passage_with_embeddings_added = os.path.join(datasets_storage_on_disk, "data",\
                                                                  "text_passages_with_embeddings_added.h5")

faiss_index_size = index.ntotal

create_file_containing_text_passages_with_embeddings(tsv_file_path1, output_path_for_text_passage_embeddings, 
                                                       chunk_size=chunk_size_for_text_passages_with_embeddings,
                                                       batch_size=batch_size_for_text_passages_with_embeddings,
                                                       max_len = passages_max_len,
                                                       output_path=output_path_for_text_passage_with_embeddings_added,
                                                       index_size = faiss_index_size )

548600
548600
548600
Loaded ALREADY exactly 1755520 rows. Stopping.
109720
CPU times: user 2min 1s, sys: 11.8 s, total: 2min 13s
Wall time: 5min 14s


In [ ]:
with h5py.File(output_path_for_text_passage_with_embeddings_added, 'r') as f:
    for key in f.keys():
        dataset = f[key]
        print(f"{key} : {dataset.shape}")

        # Print the shape of the dataset
        print("Dataset shape:", dataset.shape)

        # Read and print the first 2 rows
        first_two_rows = dataset[:2]  # Slices the first two rows
        print(type(first_two_rows[1]), (first_two_rows[1]))

dataset : (1755520,)
Dataset shape: (1755520,)
<class 'numpy.void'> (b'customers to purchase something immediately, and sharing third - party ads. email marketing has evolved rapidly alongside the technological growth of the 21st century. prior to this growth, when emails were novelties to the majority of customers, email marketing was not as effective. in 1978, gary thuerk of digital equipment corporation ( dec ) sent out the first mass email to approximately 400 potential clients via the advanced research projects agency network ( arpanet ). this email resulted in $ 13 million worth of sales in dec products, and highlighted the potential of marketing through mass emails. however, as email marketing developed as an effective means of direct communication, users began blocking out content from emails with filters and blocking programs. in order to effectively communicate a message through email, marketers had to develop a way of pushing content through to the end user, without being cu

#### <a id='toc2_1_6_2_'></a>[III- 11.2 SPLIT INTO CHUNKS OF APACHE "ARROW" FORMAT FILES THE TEXT PASSAGES AND THEIR EMBEDDINGS](#toc0_)



* THE Hugging Face's "DATASET" LIBRARY USES THE ".save_to_disk()" METHOD TO STORE IN-MEMORY DATASETS TO ARROW FORMAT

* It is crucial efficient retrieval, fast querying, and scalability because Apache Arrow is columnar and memory-mapped. Only the needed columns (e.g., embeddings) are loaded into memory

In [41]:

def break_dataset_into_chunks_in_arrow_format(hdf5_file_path, chunk_size, batch_size_chunk, output_path_for_dataset_chunks):

    with h5py.File(hdf5_file_path, 'r') as f:
        
        
        # Get the dtype of the dataset
        dtype =  f["dataset"].dtype          

        total_size = f["dataset"].shape[0]  
        print(total_size)  


        list_of_dicts_from_chunk = [] 

        chunk_num = 0
        
        for start in range(0, total_size, chunk_size):
                              
               end = min(start + chunk_size, total_size)
               
               # Read a batch of data
               data_chunk = f["dataset"][start:end]

               chunk_num +=1
               #print(chunk_num)
            
               
               # Convert the batch to a list of dictionaries
               #list_chunk = [{"data": item} for item in data_chunk]
               for item in data_chunk:
                    
                    text = item[dtype.names[0]].decode('utf-8')
                    #print(text)
                    title = item[dtype.names[1]].decode('utf-8')
                    #print(title)
                    embeddings = item[dtype.names[2]]
                    #print(embeddings)

                    row_into_dict = {"text":text, "title":title, "embeddings":embeddings}

                    list_of_dicts_from_chunk.append(row_into_dict)
                    #print("length of list_of_dicts_from_chunk:", len(list_of_dicts_from_chunk))
                    

               if len(list_of_dicts_from_chunk) >= batch_size_chunk:
                    
                    print(len(list_of_dicts_from_chunk))
                    
                    # Create a Dataset from the chunk
                    chunk_dataset = Dataset.from_list(list_of_dicts_from_chunk)
                         
                    # Convert the Dataset to a pandas DataFrame by using the 'to_pandas()' method
                    #chunk_df = chunk_dataset.to_pandas() 

                    # SAVE AS PARQUET FILE
                    #chunk_df.to_parquet( os.path.join(output_path_for_dataset_chunks, f'chunk_{chunk_num}.parquet'), index=False) 
                    chunk_dataset.save_to_disk( os.path.join(output_path_for_dataset_chunks, f'chunk_{chunk_num}'))
                    #chunk_dataset.save_to_disk( output_path_for_dataset_chunks)   
                    
                    list_of_dicts_from_chunk.clear()
                    chunk_dataset = None
                    chunk_df = None
                    del chunk_dataset, chunk_df
                    gc.collect
               
        if len(list_of_dicts_from_chunk) >0:
                    
                                        
                    print(len(list_of_dicts_from_chunk))
                    
                    # Create a Dataset from the chunk
                    chunk_dataset = Dataset.from_list(list_of_dicts_from_chunk)
                         
                    # Convert the Dataset to a pandas DataFrame by using the 'to_pandas()' method
                    #chunk_df = chunk_dataset.to_pandas() 

                    # SAVE AS PARQUET FILE
                    #chunk_df.to_parquet( os.path.join(output_path_for_dataset_chunks, f'chunk_{chunk_num}.parquet'), index=False)  
                    chunk_dataset.save_to_disk( os.path.join(output_path_for_dataset_chunks, f'chunk_{chunk_num}'))  
                    #chunk_dataset.save_to_disk( output_path_for_dataset_chunks)  

                    
               
        list_of_dicts_from_chunk.clear()
        chunk_dataset = None
        chunk_df = None
        del chunk_dataset, chunk_df
        gc.collect

                              
                    

In [48]:
%%time

output_folder_for_dataset_chunks = os.path.join(datasets_storage_on_disk,"data","chunks_of_passages_with_their_embeddings")

break_dataset_into_chunks_in_arrow_format(hdf5_file_path = output_path_for_text_passage_with_embeddings_added, 
                                          chunk_size=135040,batch_size_chunk=135040, 
                                        output_path_for_dataset_chunks = output_folder_for_dataset_chunks)
            

1755520
135040


Saving the dataset (0/2 shards):   0%|          | 0/135040 [00:00<?, ? examples/s]

135040


Saving the dataset (0/2 shards):   0%|          | 0/135040 [00:00<?, ? examples/s]

135040


...

135040


Saving the dataset (0/2 shards):   0%|          | 0/135040 [00:00<?, ? examples/s]

135040


Saving the dataset (0/2 shards):   0%|          | 0/135040 [00:00<?, ? examples/s]

CPU times: user 1min 19s, sys: 14.6 s, total: 1min 34s
Wall time: 2min 21s


In [49]:
def count_rows_in_arrow_files(folders):
    total_rows = 0
    
    for folder in folders:
        if os.path.exists(folder) and os.path.isdir(folder):

            dataset = load_from_disk(folder)  # Load the Arrow file as a dataset 
            
            total_rows += len(dataset)  # Add the number of rows to total_rows
        else:
            print(f"The folder {folder} doesn't exist or isn't a directory.")
    
    return total_rows

In [50]:

folders = [ os.path.join(output_folder_for_dataset_chunks, f"{folder}") for folder in os.listdir(output_folder_for_dataset_chunks)]

number_of_chunks_created = len(folders)                   

count_rows_in_arrow_files(folders)

1755520

In [51]:
def concatenate_arrow_files(total_chunks, chunk_folder, output_folder, batch_size=30):
    """
    Efficiently concatenates large dataset chunks to avoid memory overflow.
    
    - `batch_size`: Controls how many chunks are loaded at a time.
    - Saves intermediate results to disk to prevent excessive memory usage.
    """
    
    temp_output_base = os.path.join(output_folder, "temp")  # Base temp folder
    os.makedirs(temp_output_base, exist_ok=True)

    final_dataset = None

    for i in range(0, total_chunks, batch_size):  
        batch = []
        
        # Load a batch of chunks
        for j in range(i, min(i + batch_size, total_chunks)):
            chunk_path = os.path.join(chunk_folder, f"chunk_{j+1}")
            batch.append(load_from_disk(chunk_path))
        
        # Merge the batch
        batch_dataset = concatenate_datasets(batch)

        # Unique temp folder for each batch
        temp_output = os.path.join(temp_output_base, f"batch_{i}")
        os.makedirs(temp_output, exist_ok=True)

        # If first batch, initialize final_dataset
        if final_dataset is None:
            final_dataset = batch_dataset
        else:
            final_dataset = concatenate_datasets([final_dataset, batch_dataset]) 

        # Save intermediate dataset with unique path & delete to free memory
        final_dataset.save_to_disk(temp_output)
        del final_dataset  
        gc.collect()  # Force garbage collection
        
        # Reload from disk for next iteration
        final_dataset = load_from_disk(temp_output)  

    # Save final result
    final_dataset.save_to_disk(output_folder)

In [52]:
# Define the final dataset storage path
output_folder_for_dataset = os.path.join(datasets_storage_on_disk,"data","passages_final_dataset")


concatenate_arrow_files(total_chunks = number_of_chunks_created,
                        chunk_folder = output_folder_for_dataset_chunks,
                        output_folder = output_folder_for_dataset,
                        batch_size=10
                        )

Saving the dataset (0/12 shards):   0%|          | 0/1350400 [00:00<?, ? examples/s]

Saving the dataset (0/16 shards):   0%|          | 0/1755520 [00:00<?, ? examples/s]

Saving the dataset (0/16 shards):   0%|          | 0/1755520 [00:00<?, ? examples/s]

In [53]:
count_rows_in_arrow_files([output_folder_for_dataset])

1755520

In [412]:
# TO MAKE A SINGLE "BIG"APACHE ARROW FILE FROM ALL THE CHUNKS ARROW FILES

# from datasets import load_from_disk, concatenate_datasets
# chunk_dirs = [os.path.join(datasets_storage_on_disk, "data", "passages", f"chunk_{i+1}")  for i in range(number_of_chunks_created)]
# datasets = [load_from_disk(chunk_dir) for chunk_dir in chunk_dirs]
# # Concatenate all chunks into a single dataset
# combined_dataset = concatenate_datasets(datasets)

# combined_dataset.save_to_disk(output_folder_for_dataset_chunks)

In [ ]:
# NOT NEEDED: CONVERT FILE WITH PASSAGES TO "PICKLE" FORMAT!!!!!!!!
#pkl_file_path = os.path.join(datasets_storage_on_disk, "data","passages.pkl")
# Save the list of dictionaries to a .pkl file
#with open(pkl_file_path, 'wb') as f:
#    pickle.dump(passages_as_list_of_dict, f)

# Check if the file exists
#if not os.path.exists(pkl_file_path):
#    raise FileNotFoundError(f"The file {pkl_file_path} does not exist.")


## <a id='toc2_2_'></a>[IV- "FACEBOOK" RAG "MODEL"](#toc0_)

### <a id='toc2_2_1_'></a>[IV- 1. THE RAG "RETRIEVER"](#toc0_)

In [ ]:
device = torch.device('cuda:0')
#torch.cuda.empty_cache() 

In [ ]:
# The number of similar documents retrieved is determined by the behavior of "RagRetriever" AND By default RAG retrieves 5 documents.

In [3]:
def retriever_for_similar_passages(question, number_docs_retrieved, path_to_chunks_in_arrow_format, path_to_indeces): 

    
    rag_pretrained_model = "facebook/rag-sequence-nq"

    # INSTANTIATE THE "RAG" TOKENIZER
    tokenizer_for_retriever = RagTokenizer.from_pretrained(rag_pretrained_model) 

   # Tokenize the QUESTION
    input_ids = tokenizer_for_retriever(question, return_tensors="pt").input_ids


    device = torch.device('cuda:0')

    # INSTANTIATE THE "ENCODER" FOR THE QUESTION AND MOVE IT TO GPU
    model_result_generation = RagTokenForGeneration.from_pretrained(rag_pretrained_model).to(device)   

 
    # Get hidden states for the question USING "question_encoder" 
    # ALSO Disable gradient calculation for inference
    with torch.no_grad():  
        question_hidden_states = model_result_generation.question_encoder(input_ids.to(device))[0] 


    # Convert TENSOR to NumPy array BY MOVING IT to CPU and convert to NumPy array (cpu FRAMEWORK)
    question_hidden_states_np = question_hidden_states.detach().cpu().numpy()  

  
    # INSTANTIATE THE RAG "RETRIEVER" COMPONENT
    retriever = RagRetriever.from_pretrained(rag_pretrained_model,
                                            index_name="custom", output_retrieved=False,                                            
                                            tokenizer = tokenizer_for_retriever,
                                            n_docs= number_docs_retrieved,
                                            passages_path= path_to_chunks_in_arrow_format, 
                                            index_path=path_to_indeces
                                            )



    #  Use the retriever to retrieve relevant documents
    docs_dict = retriever(input_ids, question_hidden_states=question_hidden_states_np, return_tensors="pt")

    # RETRIEVE EMBEDDINGS OF DOCUMENTS (Assuming you have retrieved embeddings for the documents)
    retrieved_doc_embeds = docs_dict['retrieved_doc_embeds']  # Shape (n_docs, embed_dim)

    # GET CONTEXT INPUT IDs
    n_docs =docs_dict['context_input_ids'].shape[0]
    batch_size = input_ids.shape[0]
    context_input_ids = docs_dict['context_input_ids'].reshape(batch_size * n_docs, -1)


    if 'context_attention_mask' not in docs_dict:
        docs_dict['context_attention_mask'] = torch.ones_like(context_input_ids)
    #print(docs_dict['context_attention_mask'].shape)


    return question_hidden_states, retrieved_doc_embeds, model_result_generation, input_ids, \
            context_input_ids, docs_dict, tokenizer_for_retriever


In [4]:
path_to_save_faiss_indeces= os.path.join(datasets_storage_on_disk, "data","disk_index.faiss")
output_folder_for_dataset = os.path.join(datasets_storage_on_disk,"data","passages_final_dataset")

In [ ]:
# Example question
#user_prompt = "What is machine learning?"
#user_prompt = "What is the capital France?"    
#user_prompt = "what is Berlin?"    
user_prompt = "Who won the world series in 2020?"
number_docs_to_retrieve = 5 

question_hidden_states, retrieved_doc_embeds, \
model_result_generation, input_ids, context_input_ids, \
docs_dict, tokenizer_for_retriever = retriever_for_similar_passages( question = user_prompt,
                                                                     number_docs_retrieved =  number_docs_to_retrieve,             
                                                               path_to_chunks_in_arrow_format = output_folder_for_dataset, 
                                                               path_to_indeces = path_to_save_faiss_indeces
                                                            )





The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called fr

### <a id='toc2_2_2_'></a>[IV- 2. THE RAG "GENERATOR"](#toc0_)

#### <a id='toc2_2_2_1_'></a>[IV- 2.1 RESPONSE BASED ON COSINE SIMILARITY WITH RETRIEVED DOCUMENTS](#toc0_)

In [36]:
def generate_cosine_similarity_response():

    #device = torch.device('cuda:0')
    
    # Expand question_hidden_states to [1, 5, 1, 768]
    expanded_question_hidden_states = question_hidden_states.unsqueeze(1).unsqueeze(2).expand(-1, retrieved_doc_embeds.size(1), \
                                                                                              retrieved_doc_embeds.size(2), -1)

    # Compute cosine similarity along the last dimension
    doc_scores_cosine = F.cosine_similarity(expanded_question_hidden_states.to(device), retrieved_doc_embeds.to(device), dim=-1)

    # Squeeze out any unnecessary singleton dimensions for further processing
    doc_scores_cosine = doc_scores_cosine.squeeze(-1)  # Shape should now be [1, 5]


    # Generate the answer conditioned on retrieved documents
    outputs_cosine = model_result_generation.generate(
        input_ids=input_ids.to(device),
        context_input_ids=context_input_ids.to(device),
        doc_scores=doc_scores_cosine,
        context_attention_mask=docs_dict['context_attention_mask'].to(device)
    )



    # Decode the generated output
    generated_text_cosine = tokenizer_for_retriever.batch_decode(outputs_cosine, skip_special_tokens=True)[0]
    #print(generated_text_cosine)


    return generated_text_cosine

In [37]:
print(question_hidden_states.shape)
#retrieved_doc_embeds=retrieved_doc_embeds.squeeze(2)
print(retrieved_doc_embeds.shape)  # Should be (batch_size, 60, embedding_dim)
expanded_question_hidden_states = question_hidden_states.unsqueeze(1).expand(-1, retrieved_doc_embeds.size(1), -1)

print(expanded_question_hidden_states.shape )

torch.Size([1, 768])
torch.Size([1, 5, 1, 768])
torch.Size([1, 5, 768])


In [ ]:
generate_cosine_similarity_response()

' the houston astros'

### <a id='toc2_2_3_'></a>[IV- 2.2. RESPONSE BASED ON EUCLIDEAN DISTANCE WITH RETRIEVED DOCUMENTS](#toc0_)

In [39]:
def generate_euclidean_similarity_response():
        
    # Ensure that retrieved_doc_embeds is squeezed along the third dimension (singleton) before cdist
    doc_scores_euclidean = torch.cdist(question_hidden_states.unsqueeze(1).to(device), 
                                       retrieved_doc_embeds.squeeze(2).to(device), p=2)  # Shape: [1, 5]

    # Remove the extra singleton dimension if present to get shape [1, 5]
    doc_scores_euclidean = doc_scores_euclidean.squeeze(1) 



    # Generate the answer conditioned on retrieved documents
    outputs_euclidean = model_result_generation.generate(
                            input_ids=input_ids.to(device), 
                            context_input_ids=context_input_ids.to(device), 
                            doc_scores=doc_scores_euclidean,
                            context_attention_mask = docs_dict['context_attention_mask'].to(device)
                            )


    # Decode the generated output
    generated_text_euclidean = tokenizer_for_retriever.batch_decode(outputs_euclidean, skip_special_tokens=True)[0]
    #print(generated_text_euclidean)

    return generated_text_euclidean

In [ ]:
generate_euclidean_similarity_response()

' the houston astros'

### <a id='toc2_2_4_'></a>[IV- 2.3. RESPONSE BASED ON DOT PRODUCT WITH RETRIEVED DOCUMENTS](#toc0_)

In [41]:
def generate_dotproduct_similarity_response():

    # Ensure the shapes are correct before batch matrix multiplication
    question_hidden_states_unsq = question_hidden_states.unsqueeze(1) # Shape: (batch_size, 1, embed_dim)

    retrieved_doc_embeds_squeezed = retrieved_doc_embeds.squeeze(2)  # Shape: (batch_size, n_docs, embed_dim)


    # Perform batch matrix multiplication
    doc_scores_dotprod = torch.bmm(question_hidden_states_unsq.to(device), 
                                   retrieved_doc_embeds_squeezed.transpose(1, 2).to(device)).squeeze(1)  # Shape: (batch_size, n_docs)



    # Generate the answer conditioned on retrieved documents
    outputs_dotprod = model_result_generation.generate(
                            input_ids=input_ids.to(device), 
                            context_input_ids=context_input_ids.to(device), 
                            doc_scores=doc_scores_dotprod,
                            context_attention_mask = docs_dict['context_attention_mask'].to(device)
                            )


    # Decode the generated output
    generated_text_dotprod = tokenizer_for_retriever.batch_decode(outputs_dotprod, skip_special_tokens=True)[0]
    #print(generated_text_dotprod)

    return generated_text_dotprod

In [ ]:
generate_dotproduct_similarity_response()

' the houston astros'

# <a id='toc3_'></a>[X- APPENDIX](#toc0_)

In [32]:
def find_divisors(n):
    divisors = []
    for i in range(1, int(n**0.5) + 1):
        if n % i == 0:  # Check if i is a divisor
            divisors.append(i)
            if i != n // i:  # Avoid adding the square root twice if n is a perfect square
                divisors.append(n // i)
    return sorted(divisors)  # Sort the list of divisors


In [46]:

# Example usage
number = data_size
divisors_of_number1 = find_divisors(number)
print("Divisors of", number, ":", divisors_of_number1)


Divisors of 8825328 : [1, 2, 3, 4, 6, 8, 9, 12, 16, 18, 24, 27, 31, 36, 48, 54, 62, 72, 93, 108, 124, 144, 186, 216, 248, 279, 372, 432, 496, 558, 659, 744, 837, 1116, 1318, 1488, 1674, 1977, 2232, 2636, 3348, 3954, 4464, 5272, 5931, 6696, 7908, 10544, 11862, 13392, 15816, 17793, 20429, 23724, 31632, 35586, 40858, 47448, 61287, 71172, 81716, 94896, 122574, 142344, 163432, 183861, 245148, 284688, 326864, 367722, 490296, 551583, 735444, 980592, 1103166, 1470888, 2206332, 2941776, 4412664, 8825328]


In [35]:
num_vectors = index.ntotal
divisors_of_index_size= find_divisors(num_vectors)
print("\n", "Divisors of ", num_vectors, ":\n", divisors_of_index_size)


 Divisors of  1755520 :
 [1, 2, 4, 5, 8, 10, 13, 16, 20, 26, 32, 40, 52, 64, 65, 80, 104, 128, 130, 160, 208, 211, 260, 320, 416, 422, 520, 640, 832, 844, 1040, 1055, 1664, 1688, 2080, 2110, 2743, 3376, 4160, 4220, 5486, 6752, 8320, 8440, 10972, 13504, 13715, 16880, 21944, 27008, 27430, 33760, 43888, 54860, 67520, 87776, 109720, 135040, 175552, 219440, 351104, 438880, 877760, 1755520]
